In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

import sys 
sys.path.append('.')


from cgiar.utils import get_dir
from cgiar.model import XCITMultipleMLP
from cgiar.data import CGIARDataset_V4, augmentations

# reduce font size of plots
plt.rcParams.update({'font.size': 8})

In [ ]:
SEED=42
LR=1e-4
EPOCHS=30
IMAGE_SIZE=224
INITIAL_SIZE=512
TRAIN_BATCH_SIZE=128
TEST_BATCH_SIZE=64
HIDDEN_SIZE=512
NUM_FOLDS=5
NUM_VIEWS=10

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
DATA_DIR = get_dir('data/')
ARTIFACTS = get_dir('solutions/v10/#1/')

In [ ]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE),
    augmentations["RandomEqualize"],
    augmentations["RandomAffine"],
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Load test data frame from csv
X_test = pd.read_csv(DATA_DIR / 'Test.csv')

test_images = CGIARDataset_V4.load_images(X_test, DATA_DIR / "test", INITIAL_SIZE)
test_images = dict([test_images[idx] for idx in range(len(test_images))])

In [122]:
# Construct test dataset and dataloader
test_dataset = CGIARDataset_V4(
    images=test_images,
    num_views=NUM_VIEWS,
    transform=transform,
    features=X_test,
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=TEST_BATCH_SIZE, 
    shuffle=False
)

## One model

In [ ]:
fold_idx=3

model = XCITMultipleMLP(
    model_name="xcit_nano_12_p16_224",
    pretrained=True,
    num_mlps=4,
    hidden_size=HIDDEN_SIZE
)

model.load_state_dict(torch.load(ARTIFACTS / f"model_fold_{fold_idx}.pth"))

model = model.to(device)
model.eval()

test_predictions = []

with torch.no_grad():
    
    for ids, images_list, growth_stage, season, _ in tqdm(test_loader):
    
        # average predictions from all the views
        outputs = torch.stack([model((
            growth_stage.to(device).squeeze(),
            season.to(device).squeeze(),
            images.to(device)
        )) for images in images_list]).mean(dim=0)
        
        # get predictions from all the folds
        outputs = outputs.tolist()
        test_predictions.extend(list(zip(ids, outputs)))
        
# load the sample submission file and update the extent column with the predictions
submission_df = pd.read_csv('data/SampleSubmission.csv')

# update the extent column with the predictions
submission_df['extent'] = submission_df['ID'].map(dict(test_predictions))

# save the submission file and trained model
submission_df.to_csv('submission.csv', index=False)

## Ensemble Models

In [124]:
weights = {
    # 2: 0.3,
    3: 0.1
}

models = {}

for i in weights.keys():
    # initialize the model
    model = XCITMultipleMLP(
        model_name="xcit_nano_12_p16_224",
        pretrained=True,
        num_mlps=4,
        hidden_size=HIDDEN_SIZE
    )
    
    # load the model weights
    model.load_state_dict(torch.load(ARTIFACTS / f"model_fold_{i}.pth"))
    
    # move model to gpu
    model = model.to(device)
    
    # set model to evaluation mode
    model.eval()
    
    models[i] = model

test_ensemble_predictions = []


with torch.no_grad():
    for ids, images_list, growth_stage, season, _ in tqdm(test_loader):
        
        outputs = torch.zeros(len(ids)).to(device)
        
        for idx in weights.keys():
        
            # average predictions from all the views
            outputs += weights[idx] * torch.stack([model((
                growth_stage.to(device).squeeze(),
                season.to(device).squeeze(),
                images.to(device)
            )) for images in images_list]).mean(dim=0)
        
        # get predictions from all the folds
        outputs = outputs.tolist()
        test_ensemble_predictions.extend(list(zip(ids, outputs)))

# load the sample submission file and update the extent column with the predictions
submission_df = pd.read_csv('data/SampleSubmission.csv')

# update the extent column with the predictions
submission_df['extent'] = submission_df['ID'].map(dict(test_ensemble_predictions))

# save the submission file and trained model
submission_df.to_csv('submission_ensemble.csv', index=False)

 13%|█▎        | 18/136 [01:02<06:50,  3.48s/it]